In [1]:
%%javascript
require(
        ["notebook/js/outputarea"],
        function (oa) {
            oa.OutputArea.auto_scroll_threshold = -1;
            console.log("Setting auto_scroll_threshold to -1");
        });

<IPython.core.display.Javascript object>

In [56]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from urllib.parse import urlparse, parse_qsl
import matplotlib
#from matplotlib.backends.backend_pdf import FigureCanvasPdf
#matplotlib.backend_bases.register_backend('pdf', FigureCanvasPdf)
matplotlib.use('pgf')
#matplotlib.use('PS') 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import django
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
from django.db.models import Q, F, Sum, Count, FloatField, Case, When, Value, Max
import matplotlib.patches as patches
from scipy.sparse import csr_matrix, find

from scoping.models import *
from tmv_app.models import *

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize
from nltk.stem import SnowballStemmer


import textwrap as tw
from utils.text import *

doc = Doc.objects.get(pk=645877)
doc = Doc.objects.get(pk=536594)
stat = RunStats.objects.get(pk=1861)

from scipy import stats


In [58]:
doc.wtopics = Topic.objects.filter(
    run_id=stat,
    doctopic__doc=doc,
    doctopic__score__gt=stat.dt_threshold
).order_by('-doctopic__score')

doc.wtopics


<QuerySet [<Topic: {ecosystem, net, productivity}>, <Topic: {community, microbial, composition}>, <Topic: {uncertainty, estimate, parameter}>, <Topic: {trend, station, significant}>, <Topic: {stress, response, gene}>]>

In [59]:
x = range(0,6000)

tcs = Doc.objects.filter(
    query=stat.query,wosarticle__tc__isnull=False
).values('wosarticle__tc')

tcs = pd.DataFrame(list(tcs))

iptcs = Doc.objects.filter(
    query=stat.query,wosarticle__tc__isnull=False,
    ipccref__isnull=False
).values('wosarticle__tc')
iptcs = pd.DataFrame(list(iptcs))

ip_density = stats.gaussian_kde(iptcs['wosarticle__tc']).pdf(x)
all_density = stats.gaussian_kde(tcs['wosarticle__tc']).pdf(x)

In [60]:
doc.dtopics = DynamicTopic.objects.filter(
    run_id=stat,topicdtopic__topic__in=doc.wtopics,
    topicdtopic__topic__doctopic__doc=doc,
).annotate(
    topic_combination=F('topicdtopic__score')*F('topicdtopic__topic__doctopic__score')
).filter(topic_combination__gt=stat.dt_threshold).order_by('-topic_combination')


#bla = doc.dtopics.values('title').aggregate(
#    bla=Sum('topic_combination')
#)

doc.dtopics

<QuerySet []>

In [61]:
doc.dtopics.distinct().count()

0

In [62]:
if stat.method=="DT":
    df = pd.DataFrame.from_dict(list(doc.dtopics.values(
        'id','title',
        'topicdtopic__topic__title',
        'topicdtopic__topic__id',
        'topic_combination',
        'topicdtopic__score'
    ))).sort_values('title')
    df['scaled_link'] = df['topicdtopic__score']/df['topicdtopic__score'].max()
    df.head()
else:
    df = None

In [63]:
tdf = pd.DataFrame.from_dict(list(doc.wtopics.values(
    'id','title','doctopic__score'
)))

tdf['sum']=tdf['doctopic__score'].sum()
tdf['share']=tdf['doctopic__score']/tdf['sum']*100
tdf['x'] = tdf['share'].cumsum().shift().fillna(0)
tdf['center'] = tdf['x']+tdf['share']/2

tdf.head()

,doctopic__score,id,title,sum,share,x,center
0,0.021950,170519,"{ecosystem, net, productivity}",0.06652,32.997962,0.000000,16.498981
1,0.016929,170497,"{community, microbial, composition}",0.06652,25.449860,32.997962,45.722892
2,0.010808,170510,"{uncertainty, estimate, parameter}",0.06652,16.247644,58.447822,66.571644
3,0.008570,170508,"{trend, station, significant}",0.06652,12.883130,74.695466,81.137030
4,0.008263,170436,"{stress, response, gene}",0.06652,12.421405,87.578595,93.789298


In [64]:
if stat.method=="DT":
    dtdf = df.groupby(['title','id']).sum().sort_values('topic_combination',ascending=False).reset_index()
    dtdf['sum']=dtdf['topic_combination'].sum()
    dtdf['share']=dtdf['topic_combination']/dtdf['sum']*100
    dtdf['x'] = dtdf['share'].cumsum().shift().fillna(0)
    dtdf['center'] = dtdf['x']
    dtdf['share']/2

    dtdf.head()
else:
    dtdf = None


In [70]:
if stat.fancy_tokenization:
    ######################################
    ## A fancy tokenizer
    
    ndocs = Doc.objects.filter(query=stat.query).count()

    from nltk import wordpunct_tokenize
    from nltk import WordNetLemmatizer
    from nltk import sent_tokenize
    from nltk import pos_tag
    from nltk.corpus import stopwords as sw
    punct = set(string.punctuation)
    from nltk.corpus import wordnet as wn
    stopwords = set(sw.words('english'))

    if stat.extra_stopwords:
        stopwords = stopwords | set(stat.extra_stopwords)

    def lemmatize(token, tag):
            tag = {
                'N': wn.NOUN,
                'V': wn.VERB,
                'R': wn.ADV,
                'J': wn.ADJ
            }.get(tag[0], wn.NOUN)
            return WordNetLemmatizer().lemmatize(token, tag)

    kws = Doc.objects.filter(
        query=stat.query,
        kw__text__iregex='\w+[\-\ ]'
    ).values('kw__text').annotate(
        n = Count('pk')
    ).filter(n__gt=ndocs//200).order_by('-n')

    kw_text = set([x['kw__text'].replace('-',' ') for x in kws])
    kw_ws = set([x['kw__text'].replace('-',' ').split()[0] for x in kws]) - stopwords

    def fancy_tokenize(X):

        common_words = set([x.lower() for x in X.split()]) & kw_ws
        for w in list(common_words):
            w = w.replace('(','').replace(')','')
            wpat = "({}\W*\w*)".format(w)
            wn = [x.lower().replace('-',' ') for x in re.findall(wpat, X, re.IGNORECASE)]
            kw_matches = set(wn) & kw_text
            if len(kw_matches) > 0:
                for m in kw_matches:
                    print(m)
                    insensitive_m = re.compile(m, re.IGNORECASE)
                    X = insensitive_m.sub(' ', X)
                    yield m.replace(" ","-")

        for sent in sent_tokenize(X):
            for token, tag in pos_tag(wordpunct_tokenize(sent)):
                token = token.lower().strip()
                if token in stopwords:
                    continue
                if all(char in punct for char in token):
                    continue
                if len(token) < 3:
                    continue
                if all(char in string.digits for char in token):
                    continue
                lemma = lemmatize(token,tag)
                yield lemma

    tokenizer = fancy_tokenize
else:
    tokenizer = snowball_stemmer()

In [72]:
def doc_term_frequency(doc, tokenizer):
    
    count_vectorizer = CountVectorizer(
        max_df=1,
        min_df=0,
        ngram_range=(1,1),
        tokenizer=tokenizer,
        stop_words=stoplist
    )

    tfidf = count_vectorizer.fit_transform([doc.content])
    vocab = count_vectorizer.get_feature_names()

    freqs = find(tfidf)[2]
    most_frequent = np.argsort(-freqs)
    return(vocab,most_frequent,freqs)

In [76]:
def topic_dtopic_graph(ax,colours,tdf,dtdf,df):
    #plt.rcParams["figure.figsize"] = [12,7]
    row_2 = 1.5
    ax.set_xlim([0,100])
    ax.set_ylim([0,row_2+1.1])
    
    topics = {}
    dtopics = {}
    if dtdf is None:
        ax.set_ylim([0,1.1])
        for i, row in tdf.iterrows():
            if i >= len(colours):
                continue
            else:
                c = colours[i]
            topics[row['id']] = row
            topics[row['id']]['colour'] = c
            t = doc.wtopics.filter(pk=row['id']).first()
            t.colour=c
            ax.add_patch(patches.Rectangle(
                (row['x'],0),
                width=row['share'],
                height=1,
                fill=True,
                #figure=fig,
                #alpha=0.5,
                edgecolor='black',
                facecolor=c
            ))
            ax.text(
                row['center'],0.9,row['title'],rotation=270,
                ha='center'
            )        
    else:
        for i, row in tdf.iterrows():
            topics[row['id']] = row

            ax.add_patch(patches.Rectangle(
                (row['x'],0),
                width=row['share'],
                height=1,
                fill=True,
                #figure=fig,
                alpha=0.5,
                edgecolor='black'
            ))
            ax.text(
                row['center'],0.9,row['title'],rotation=270,
                ha='center'
            )

        for i, row in dtdf.iterrows():
            if i >= len(colours):
                continue
            else:
                c = colours[i]
            dtopics[row['id']] = row
            dtopics[row['id']]['colour'] = c
            dt = doc.dtopics.filter(pk=row['id']).first()
            dt.colour=c
            ax.add_patch(patches.Rectangle(
                (row['x'],row_2),
                width=row['share'],
                height=1,
                fill=True,
                #figure=fig,
                #alpha=0.5,
                edgecolor='black',
                facecolor=c
            ))   
            if row['share'] > 4:
                ax.text(
                    row['center'],row_2 +.9,row['title'],rotation=270,
                    ha='center'
                )

        for i, row in df.iterrows():
            try:
                dt = dtopics[row['id']]
            except:
                continue
            t = topics[row['topicdtopic__topic__id']]
            #ax.plot(
            #    [t['center'],dt['center']],[1,row_2],
            #    alpha=row['scaled_link']+0.5,
            #    linewidth=row['scaled_link']*5,
            #    color=dt['colour']
            #)
            ax.fill(
                [t['x'],t['x']+t['share'],dt['x']+dt['share'],dt['x']],
                [1,1,row_2,row_2],
                alpha=0.5,
                color="grey"
            )
        ax.axis('off')

        ax.text(50,-0.1,"\\textbf{Window topics in $doc_1$}",ha="center",va="center")
        ax.text(50,row_2+1.1,"\\textbf{Dynamic topics in $doc_1$}",ha="center",va="center")

       
    return [topics,dtopics]

gridspec = (3,6)

#plt.close('all')
fig = plt.figure(figsize=(12,7))
plt.axis('off')

axs = [
    plt.subplot2grid(gridspec,(0,0),colspan=3,rowspan=2),
    plt.subplot2grid(gridspec,(0,3),colspan=3,rowspan=2),
    plt.subplot2grid(gridspec,(2,0),colspan=2,rowspan=1),
    plt.subplot2grid(gridspec,(2,2),colspan=2,rowspan=1),
    plt.subplot2grid(gridspec,(2,4),colspan=2,rowspan=1)
]
plt.tight_layout()

for ax in axs:
    ax.grid(False)
    ax.axis('off')

# Topic-dtopic graph

colours = [
    '#8dd3c7','#ffffb3','#bebada',
    '#fb8072','#80b1d3','#fdb462',
    '#b3de69','#fccde5','#d9d9d9',
    '#bc80bd','#ccebc5','#ffed6f'
]

# Dynamic topics-window topics
topics, dtopics = topic_dtopic_graph(axs[0],colours,tdf,dtdf,df)



# Docs
doc.wrapped_content = get_wrapped_content(doc,57,dtopics, topics)
tb, tb_coords = draw_docs(axs[1],fig,doc,9)

## Doc Topic Table
doc.vocab, doc.most_frequent, doc.freqs = doc_term_frequency(doc, tokenizer)
dt_cells = doctopic_table(doc,5,5)
axs[2].table(
    cellText=dt_cells,
    bbox=[0,0,1,1]
)
axs[2].text(0.5,-0.05,"\\textbf{Doc Topic Matrix}",va="top",ha="center")
axs[2].text(1.045,0.5,"$\\times$",va="center",ha="center")


## Topic Term Table
tt_cells = topicterm_table(doc,5,5)
axs[3].table(
    cellText=tt_cells,
    bbox=[0,0,1,1]
)
axs[3].text(0.5,-0.05,"\\textbf{Topic Term Matrix}",va="top",ha="center")
axs[3].text(1.04,0.5,"$\\approx$",va="center",ha="center")



cells = docterm_table(doc,5,5)
axs[4].table(
    cellText=cells,
    bbox=[0,0,1,1]
)
axs[4].text(0.5,-0.05,"\\textbf{Doc Term Matrix}",va="top",ha="center")

fig.patch.set_facecolor('#f0f0f0')

plt.savefig("../plots/single_doc_3_{}_{}.pdf".format(doc.id, stat.pk),facecolor=fig.get_facecolor())
        

/home/galm/software/django/tmv/venv/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


<QuerySet [<Topic: {ecosystem, net, productivity}>, <Topic: {community, microbial, composition}>, <Topic: {uncertainty, estimate, parameter}>, <Topic: {trend, station, significant}>, <Topic: {stress, response, gene}>]>
ecological
ecosystems
community
uncertainty
community
ecosystem
ecological
trends
ecological
climate change
global change
global warming
['ecosystem', ' net', ' productivity']
['community', ' microbial', ' composition']
['uncertainty', ' estimate', ' parameter']


In [51]:
def get_wrapped_content(doc,w,dtopics,wtopics=None):
    snowball_stemmer = SnowballStemmer("english")
    wrapped_content=""       
    if wtopics:
        topics = doc.wtopics
        dtopics = wtopics
    else:
        topics = doc.wtopics
        
    print(topics)
    for word in tw.fill(tw.dedent(doc.content.rstrip()),width=w).split(' '):
        for i, word in enumerate(word.split('\n')):           
            wordadd = word
            for topic in topics:    
                if snowball_stemmer.stem(word) in topic.top_words or word in topic.top_words:
                    print(word)
                    try:
                        dt = dtopics[topic.id]
                    except:
                        continue
                    #wordadd='\\textcolor{green}{'+word+'}'
                    wordadd='\\colorbox[HTML]{'+dt['colour'].replace('#','').upper()+'}{'+word+'}'
            if i > 0:
                wrapped_content+='\n'
            wrapped_content+=wordadd+" "

    title = tw.fill(tw.dedent(doc.title.rstrip()),width=w)

    wrapped_content = '\\textbf{'+title+'} \n\n '+wrapped_content
    return wrapped_content

get_wrapped_content(doc,57,dtopics, wtopics=topics)


<QuerySet [<Topic: {ecosystem, net, productivity}>, <Topic: {community, microbial, composition}>, <Topic: {uncertainty, estimate, parameter}>, <Topic: {trend, station, significant}>, <Topic: {stress, response, gene}>, <Topic: {specie, native, invasive}>, <Topic: {system, performance, hybrid}>, <Topic: {change, climatic, shift}>, <Topic: {record, late, early}>, <Topic: {landscape, erosion, disturbance}>, <Topic: {environmental, impact, life}>, <Topic: {research, science, issue}>, <Topic: {sea, level, rise}>, <Topic: {fish, fishery, marine}>, <Topic: {pacific, sst, enso}>, <Topic: {global, earth, world}>, <Topic: {human, activity, natural}>, <Topic: {arctic, permafrost, warm}>, <Topic: {ozone, stratospheric, tropospheric}>]>
ecological
ecological
impacts
impacts
terrestrial
tropical
marine
ecosystems
community
uncertainty
community
ecosystem
global
review
pattern
ecological
ecological
change
early
trends
global
ecological
ecological
change


'\\textbf{Ecological responses to recent climate change} \n\n There is now ample evidence of the \\colorbox[HTML]{BC80BD}{ecological} \\colorbox[HTML]{CCEBC5}{impacts} of \nrecent climate change, from polar terrestrial to tropical \nmarine environments. The responses of both flora and \nfauna span an array of \\colorbox[HTML]{8DD3C7}{ecosystems} and organizational \nhierarchies, from the species to the \\colorbox[HTML]{FFFFB3}{community} levels. \nDespite continued \\colorbox[HTML]{BEBADA}{uncertainty} as to \\colorbox[HTML]{FFFFB3}{community} and \n\\colorbox[HTML]{8DD3C7}{ecosystem} trajectories under global change, our \\colorbox[HTML]{FFED6F}{review} \nexposes a coherent \\colorbox[HTML]{FCCDE5}{pattern} of \\colorbox[HTML]{BC80BD}{ecological} \\colorbox[HTML]{FCCDE5}{change} across \nsystems. Although we are only at an \\colorbox[HTML]{D9D9D9}{early} stage in the \nprojected \\colorbox[HTML]{FB8072}{trends} of global warming, \\colorbox[HTML]{BC80BD}{ecological} responses \nto rec

In [8]:
def draw_docs(ax,fig,doc,fs=8):
    ax.grid(False)
    ax.axis('off')   
    pad = 0.065
    def draw_box(ax,fig,doc,pad,x=0,y=0):
        r = fig.canvas.get_renderer()
        transf = ax.transData.inverted()   
        tb = ax.text(
            x+pad,
            y+pad,
            doc.wrapped_content,
            va="bottom",
            fontsize=fs
        )
        tb_extent = tb.get_window_extent(renderer=r)
        tb_coords = tb_extent.transformed(transf)

        ax.add_patch(patches.Rectangle(
            (tb_coords.x0-pad,tb_coords.y0-pad),
            width=tb_coords.x1-tb_coords.x0+pad*2,
            height=tb_coords.y1-tb_coords.y0+pad*2,
            fill=True,
            #figure=fig,
            #alpha=0.5,
            edgecolor='black',
            facecolor='white'
        )) 
        return [tb,tb_coords]

    tb, tb_coords = draw_box(ax,fig,doc,pad)

    pos_shuffle=pad

    for i in [0,1,2]:
        if i > 0:
            ax.add_patch(patches.Rectangle(
                (tb_coords.x0-pad+pos_shuffle*i,tb_coords.y0-pad+pos_shuffle*i),
                width=tb_coords.x1-tb_coords.x0+pad*2,
                height=tb_coords.y1-tb_coords.y0+pad*2,
                fill=True,
                edgecolor='black',
                facecolor='white',
                zorder=-i
            )) 
        
        dbox = ax.text(
            tb_coords.x0-pad/2+pos_shuffle*i,
            tb_coords.y1+pad/2+pos_shuffle*i,
            "$doc_{}$".format(i+1),
            va="center"
        ) 
        
    return [tb,tb_coords]

In [9]:
def docterm_table(doc,rows,cols):
    cells = []
    terms = []
    for i in range(rows):
        row = []
        #row_names.append("doc")
        for j in range(cols):
            if j==0:
                if i==0:
                    row.append("Term\nDoc")
                else:
                    row.append("$doc_{}$".format(i))
                terms.append("Term\nDoc")
            else:
                w = doc.most_frequent[j-1]
                if i == 0:
                    row.append(doc.vocab[w])
                elif i==1:
                    row.append(doc.freqs[w])
                else:
                    row.append("...")
        cells.append(row)
    return cells

def split_title(title):
    return "{}\n{}\n{}".format(*title.replace('}','').replace('{','').split(','))

def topicterm_table(doc,rows,cols):
    tt_cells = []
    for i in range(rows-1):
        tt_row = []
        for j in range(cols):
            if i ==0:
                freq_subset = doc.most_frequent[:cols-1]
                tt_row = ["Term\nTopic"] + [doc.vocab[w] for w in freq_subset]
            else:
                t = doc.wtopics[i-1]
                if j ==0:
                    title = t.title.replace('}','').replace('{','').split(',')
                    print(title)
                    tt_row.append("{}\n{}\n{}".format(*title))
                    #tt_row.append("warm\nglobal")
                    #tt_row.append(t.title.replace(',','\n'))
                else:
                    w = doc.most_frequent[j-1]
                    term = Term.objects.get(title=doc.vocab[w])
                    try:
                        tt = round(TopicTerm.objects.get(topic=t,term=term).score,2)
                    except:
                        tt = 0
                    tt_row.append(tt)

        tt_cells.append(tt_row)
    return tt_cells

def doctopic_table(doc,rows,cols):
    dt_cells = []
    for i in range(rows-1):
        dt_row = []
        for j in range(cols):
            if i ==0:
                dt_row = ["Topic\nDoc"] + [split_title(t.title) for t in doc.wtopics[:cols-1]]
            else:      
                if j ==0:
                    dt_row.append("$doc_{}$".format(i))
                else:
                    t = doc.wtopics[j-1]
                    if i==1:
                        dt = round(DocTopic.objects.get(topic=t,doc=doc).score,3)
                    else:
                        dt = "..."
                    dt_row.append(dt)

        dt_cells.append(dt_row)
    return dt_cells

In [34]:
## Set up latex

pgf_with_latex = {
    "text.usetex": True,            # use LaTeX to write all text
    "pgf.rcfonts": False,           # Ignore Matplotlibrc
    "text.latex.unicode": True,
    "pgf.preamble": [
        #r"\usepackage[utf8x]{inputenc}",
        r"\usepackage{xcolor}"
    ],
    "pgf.texsystem" : "xelatex",
    "figure.figsize": [12,7]
}
matplotlib.rcParams.update(pgf_with_latex)

#set up grid
gridspec = (3,6)

#plt.close('all')
fig = plt.figure(figsize=(12,7))
plt.axis('off')

axs = [
    plt.subplot2grid(gridspec,(0,0),colspan=3,rowspan=2),
    plt.subplot2grid(gridspec,(0,3),colspan=3,rowspan=2),
    plt.subplot2grid(gridspec,(2,0),colspan=2,rowspan=1),
    plt.subplot2grid(gridspec,(2,2),colspan=2,rowspan=1),
    plt.subplot2grid(gridspec,(2,4),colspan=2,rowspan=1)
]
plt.tight_layout()

for ax in axs:
    ax.grid(False)
    ax.axis('off')

# Topic-dtopic graph

colours = [
    '#8dd3c7','#ffffb3','#bebada',
    '#fb8072','#80b1d3','#fdb462',
    '#b3de69','#fccde5','#d9d9d9',
    '#bc80bd','#ccebc5','#ffed6f'
]

# Dynamic topics-window topics
topics, dtopics = topic_dtopic_graph(axs[0],colours,tdf,dtdf,df)



# Docs
doc.wrapped_content = get_wrapped_content(doc,57,dtopics)
tb, tb_coords = draw_docs(axs[1],fig,doc,9)

## Doc Topic Table
doc.vocab, doc.most_frequent, doc.freqs = doc_term_frequency(doc)
dt_cells = doctopic_table(doc,5,5)
axs[2].table(
    cellText=dt_cells,
    bbox=[0,0,1,1]
)
axs[2].text(0.5,-0.05,"\\textbf{Doc Topic Matrix}",va="top",ha="center")
axs[2].text(1.045,0.5,"$\\times$",va="center",ha="center")


## Topic Term Table
tt_cells = topicterm_table(doc,5,5)
axs[3].table(
    cellText=tt_cells,
    bbox=[0,0,1,1]
)
axs[3].text(0.5,-0.05,"\\textbf{Topic Term Matrix}",va="top",ha="center")
axs[3].text(1.04,0.5,"$\\approx$",va="center",ha="center")



cells = docterm_table(doc,5,5)
axs[4].table(
    cellText=cells,
    bbox=[0,0,1,1]
)
axs[4].text(0.5,-0.05,"\\textbf{Doc Term Matrix}",va="top",ha="center")

fig.patch.set_facecolor('#f0f0f0')

plt.savefig("../plots/single_doc_3_{}.pdf".format(doc.id),facecolor=fig.get_facecolor())

['ecosystem', ' net', ' productivity']
['community', ' microbial', ' composition']
['uncertainty', ' estimate', ' parameter']


In [13]:
gridspec = (3,6)

#plt.close('all')
fig = plt.figure(figsize=(12,7))
plt.axis('off')

axs = [
    plt.subplot2grid(gridspec,(0,0),colspan=3,rowspan=2),
    plt.subplot2grid(gridspec,(0,3),colspan=3,rowspan=2),
    plt.subplot2grid(gridspec,(2,0),colspan=3,rowspan=1),
    plt.subplot2grid(gridspec,(2,3),colspan=3,rowspan=1)
]

for ax in axs:
    ax.grid(False)
    ax.axis('off')

    
# Dynamic topics-window topics
#topics, dtopics = topic_dtopic_graph(axs[0],colours,tdf,dtdf,df)



# Docs
doc.wrapped_content = get_wrapped_content(doc,57,topics)
tb, tb_coords = draw_docs(axs[1],fig,doc,9)

## Doc Topic Table
doc.vocab, doc.most_frequent, doc.freqs = doc_term_frequency(doc)

## Topic Term Table
tt_cells = topicterm_table(doc,5,5)
axs[2].table(
    cellText=tt_cells,
    bbox=[0,0,1,1]
)
axs[2].text(0.5,-0.05,"\\textbf{Topic Term Matrix}",va="top",ha="center")



cells = docterm_table(doc,5,5)
axs[3].table(
    cellText=cells,
    bbox=[0,0,1,1]
)
axs[3].text(0.5,-0.05,"\\textbf{Doc Term Matrix}",va="top",ha="center")

fig.patch.set_facecolor('#f0f0f0')
plt.savefig("../plots/single_doc_2_{}_{}.pdf".format(doc.id,stat.id),facecolor=fig.get_facecolor())

NameError: name 'topics' is not defined

In [ ]:
# 1 Table
gridspec = (3,6)

#plt.close('all')
fig = plt.figure(figsize=(12,7))
plt.axis('off')

axs = [
    plt.subplot2grid(gridspec,(0,0),colspan=3,rowspan=3),
    plt.subplot2grid(gridspec,(0,3),colspan=3,rowspan=2),
    plt.subplot2grid(gridspec,(2,3),colspan=3,rowspan=1)
]

for ax in axs:
    ax.grid(False)
    ax.axis('off')

    
# Dynamic topics-window topics
topics, dtopics = topic_dtopic_graph(axs[0],colours,tdf,dtdf,df)
# Docs
tb, tb_coords = draw_docs(axs[1],fig,doc,9)

cells = docterm_table(doc,5,5)
axs[2].table(
    cellText=cells,
    bbox=[0,0,1,1]
)
axs[2].text(0.5,-0.05,"\\textbf{Doc Term Matrix}",va="top",ha="center")

fig.patch.set_facecolor('#f0f0f0')
plt.savefig("../plots/single_doc_1_{}.pdf".format(doc.id),facecolor=fig.get_facecolor())

In [ ]:
plt.rcParams["figure.figsize"] = [3,7.5]

fig, ax = plt.subplots()

p1, = ax.plot(all_density,x, label="All Documents")
p2, = ax.plot(ip_density,x, label="Cited in IPCC")

plt.legend(handles=[p1,p2])

ax.invert_yaxis()
plt.yscale('log')

plt.show()

In [ ]:
# 1 Table
gridspec = (18,18)

#plt.close('all')
fig = plt.figure(figsize=(12,7))
plt.axis('off')

axs = [
    plt.subplot2grid(gridspec,(0,0),colspan=8,rowspan=18),
    plt.subplot2grid(gridspec,(0,8),colspan=8,rowspan=12),
    plt.subplot2grid(gridspec,(12,8),colspan=8,rowspan=6),
    plt.subplot2grid(gridspec,(3,16),colspan=2,rowspan=15)
]

for ax in axs[0:2]:
    ax.grid(False)
    ax.axis('off')

    
# Dynamic topics-window topics
topics, dtopics = topic_dtopic_graph(axs[0],colours,tdf,dtdf,df)
# Docs
tb, tb_coords = draw_docs(axs[1],fig,doc,9)

cells = docterm_table(doc,5,5)
axs[2].table(
    cellText=cells,
    bbox=[0,0,1,1]
)
axs[2].text(0.5,-0.05,"\\textbf{Doc Term Matrix}",va="top",ha="center")

ax = axs[3]
ax.plot(all_density,x, label="All Documents")
ax.plot(ip_density,x, label="Cited in IPCC")



#ax.legend(handles=[p1,p2])
ax.xaxis.tick_top()
ax.yaxis.tick_right()
ax.invert_yaxis()
ax.set_yscale('log')
ax.axhline(doc.wosarticle.tc)
ax.set_title("Citations",y=1.08)

fig.patch.set_facecolor('#f0f0f0')
plt.savefig("../plots/single_doc_1_citation_{}.pdf".format(doc.id),facecolor=fig.get_facecolor())